Regression with orthogonal projector/matrices
================================================================

In this example we explain how the `use_orthogonal_projector` option in `skcosmo.linear_model.OrthogonalRegression` can result in nonanalytic behavior for predictions with respect to changes in the feature dimension.
`skcosmo.linear_model.OrthogonalRegression` solves the linear regression problem

$$ \min_\Omega ||y - X\Omega\||_F$$

with the constraint that $\Omega$ is an orthogonal projection/matrix  $\Omega^T\Omega=I$. In this formulation it can be only solved if $y$ and $X$ have the same dimension, i.e. they agree in the number of features. If `use_orthogonal_projector` is set to False, the feature dimension of $X$ or $y$ are padded with 0 values such that they agree in the number of dimension. To be able to compute an orthogonal projector without any padding of the matrices the option `use_orthogonal_projector` is offered with the caveat that it does not always well behave analytically. For this approach we take the linear regression solution $W$

$$ \min_W ||y - XW\||_F,$$

applyt SVD on it $W = USV^T$ and solve the Procrustes problem for

$$\min_\Omega' ||yV - XU\Omega'\||_F\quad \Omega'^T\Omega'=I$$

The final orthogonal projector is then $\Omega =  U\Omega' V^T$. This solution can however be problematic, since changing the number of features can result in different predictions. This notebook shows an example of this problematic behaviour. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skcosmo.linear_model import OrthogonalRegression

# These are coordinates of a 3-dimensional cube. We treat the points of the cube as samples
# and the 3 dimensions as features x y z
cube = np.array(
    [
        #x  y  z
        [0, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 1, 0],
        [0, 1, 1],
        [1, 0, 1],
        [1, 1, 1],
    ]
)


# the x y coordinates of the cube
xy_plane_projected_cube = cube[:, [0, 1]]

# cube with 
def z_scaled_cube(z_scaling):
    return np.array(
        [
            [0, 0, 0],
            [1, 0, 0],
            [0, 1, 0],
            [0, 0, z_scaling],
            [1, 1, 0],
            [0, 1, z_scaling],
            [1, 0, z_scaling],
            [1, 1, z_scaling],
        ]
    )

# In terms of regressable information `xy_plane_projected_cube` can be seen as z_scaled_cube with z_scaling = 0.
# Adding features with zero values should in general not change the prediction quality

We compute now the orthogonal regression error fitting on a cube with z-scaling in the range of [0,1] to predict the cube. For the case of a zero z-scaling the error is computed one time with a third dimension and one time without it (using `xy_plane_projected_cube`). The regression is done with `skcosmo.linear_model.OrthogonalRegression` by first setting the `use_orthogonal_projector` to True.

In [ ]:
z_scalings = np.linspace(0, 1, 10)

regression_errors_for_z_scaled_cube_using_orthogonal_projector = []
orth_reg_using_orthogonal_projector = OrthogonalRegression(use_orthogonal_projector=True)
for z in z_scalings:
    orth_reg_using_orthogonal_projector.fit(cube, z_scaled_cube(z))
    orth_reg_pred_cube = orth_reg_using_orthogonal_projector.predict(cube)
    regression_error = np.linalg.norm(z_scaled_cube(z) - orth_reg_pred_cube)
    regression_errors_for_z_scaled_cube_using_orthogonal_projector.append(regression_error)


orth_reg_using_orthogonal_projector.fit(cube, xy_plane_projected_cube)
orth_reg_pred_cube = orth_reg_using_orthogonal_projector.predict(cube)
regression_error_for_xy_plane_projected_cube_using_orthogonal_projector = (
        np.linalg.norm(xy_plane_projected_cube - orth_reg_pred_cube)
    )

Now we set `use_orthogonal_projector` to False and repeat the above regression.

In [ ]:
orth_reg = OrthogonalRegression(use_orthogonal_projector=False)
regression_errors_for_z_scaled_cube_zero_padded = []
for z in z_scalings:
    orth_reg.fit(cube, z_scaled_cube(z))
    orth_reg_pred_cube = orth_reg.predict(cube)
    regression_error = np.linalg.norm(z_scaled_cube(z) - orth_reg_pred_cube)
    regression_errors_for_z_scaled_cube_zero_padded.append(regression_error)

Setting the `use_orthogonal_projector` option to False pads automatically input and output data to the same dimension with zeros. Therefore we pad `xy_plane_projected_cube` to three dimensions with zeros.

In [ ]:
orth_reg.fit(cube, xy_plane_projected_cube)
orth_reg_pred_cube = orth_reg.predict(cube)
zero_padded_xy_plane_projected_cube = np.pad(xy_plane_projected_cube, [(0, 0), (0, 1)])

regression_error_for_xy_plane_projected_cube_zero_padded = np.linalg.norm(
    zero_padded_xy_plane_projected_cube - orth_reg_pred_cube
)

Now we plot our results

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 3.8), sharey=True)

axes[0].plot(
    z_scalings,
    regression_errors_for_z_scaled_cube_using_orthogonal_projector,
    label="Regression error for z-scaled cube",
)
axes[0].scatter(
    0,
    regression_error_for_xy_plane_projected_cube_using_orthogonal_projector,
    label="Regression error for xy_plane_projected_cube",
)
axes[0].set_title(
    "Orthogonal regression error for\n features using orthogonal projector\n (use_orthogonal_projector=True)"
)
axes[0].set_xlabel("scaling in z direction")
axes[0].set_ylabel("orthogonal regression error")

axes[1].plot(
    z_scalings,
    regression_errors_for_z_scaled_cube_zero_padded,
    label="Regression error for z-scaled cube",
)
axes[1].scatter(
    0,
    regression_error_for_xy_plane_projected_cube_zero_padded,
    label="Regression error for xy_plane_projected_cube",
)
axes[1].set_title("Orthogonal regression error for\n zero padded features\n (use_orthogonal_projector=False) ")
axes[1].set_xlabel("scaling in z direction")
axes[1].legend(loc="lower right")
plt.show()

It can be seen that setting `use_orthogonal_projector` to True, the regression error does not behave analytically with respect to changes in the dimension, since the regression error using `xy_plane_projected_cube` has an abrupt jump in contrast to retaining the third dimension with 0 values. When setting `use_orthogonal_projector` to False this nonanalytic behavior is not present, since it uses the padding solution. The recommendation is therefore to set `use_orthogonal_projector` to False. This however is not always the intended usage, since it changes the nature of $X$ and $y$ by padding it with 0 values, so we also offer the other approach.